In [1]:
import matlab.engine
import socket, struct
import control as ctrl
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import integrate
import torch.nn.functional as F
import threading
import concurrent.futures
# import subprocess
# import os
# import signal


In [2]:
## matlab api connection
eng = matlab.engine.start_matlab()
eng.cd(r'C:\Users\pvm8318\Documents\NeoVim\Reinforcement')
eng.addpath(r'C:\Users\pvm8318\Documents\NeoVim\Reinforcement')
def SimRun():
    eng.sim('Buck_Converter.slx')
    return

In [3]:
## TCP Connection
MESSAGE_SIZE = 24
DELIMITER = b'\n'
TCP_IP = '156.62.139.28'
TCP_PORT = 50000
BUFFER_SIZE = MESSAGE_SIZE if MESSAGE_SIZE else 32  # Minimum for two doubles


def send_data(conn, val):
    """Sends two double-precision numbers."""
    # Fixed Size
    msg = struct.pack('>d', val)
    conn.send(msg)

def receive_data(conn):
    """Receives three double-precision numbers."""
    if MESSAGE_SIZE:
        data = conn.recv(MESSAGE_SIZE)
        val1, val2, Time = struct.unpack('>ddd', data)
    else:
        # Delimiter
        val1 = None
        val2 = None
        Time = None
        while True:
            data = conn.recv(BUFFER_SIZE)
            if DELIMITER in data:
                val1_bytes, remaining = data.split(DELIMITER, 1)
                val1 = struct.unpack('>d', val1_bytes)[0]
                if DELIMITER in remaining:
                    val2_bytes, time_bytes = remaining.split(DELIMITER, 1)
                    val2 = struct.unpack('>d', val2_bytes)[0]
                    Time = struct.unpack('>d', time_bytes)[0]
                    break
    return val1, val2, Time

# Close the existing socket connection if it is open




In [4]:
## Buck converter parameters 
Vref = 5
u = 0
R = 1.0  # Resistance
L = 0.1  # Inductance
C = 1e-3  # Capacitance
Vin = 12.0  # Input voltage
Vref = 5.0  # Reference output voltage.0
# State-space representation of the buck converter
A = np.array([[0, 1 / C], [-1 / L, -R / L]])
B = np.array([[0], [1 / L]])
#steady state calculation
duty_cycle =Vref/Vin
Iout = Vref/R
ILref = Iout/duty_cycle

In [5]:
# # RL Configuration
input_size = 2
hidden_size = 128
output_size = 1
F = nn.functional
class Agent(nn.Module):
        def __init__(self):
                super(Agent, self).__init__()
                self.layer1 = nn.Linear(input_size, hidden_size)
                self.layer2 = nn.Linear(hidden_size, output_size)

        def forward(self, x):
                x = F.relu(self.layer1(x))
                x = self.layer2(x)
                return x


agent = Agent()

# reward calculation
def reward(x, u):
        V = x[0]
        IL = x[1]
        Q = 10*np.eye(2)  # State penalty matrix
        R = 1 
        reward = -np.linalg.norm(x - np.array([Vref, ILref]))**2 
        # reward = -np.linalg.norm(x - np.array([Vref, ILref]))**2 - u**2 * R
        return reward

# RL Agent 
class RandomPolicy(nn.Module):
        def forward(self, state):
                # Random policy: sample a random action
                return torch.rand(1)
        
# Control law
def control_law(x):
        with torch.no_grad():
                action = agent(torch.tensor(x, dtype=torch.float32))
        return action.item()  


In [18]:
# RL Configuration
# class Actor(nn.Module):
#     def __init__(self, state_dim, action_dim, max_action):
#         super(Actor, self).__init__()

#         self.l1 = nn.Linear(state_dim, 400)
#         self.l2 = nn.Linear(400, 300)
#         self.l3 = nn.Linear(300, action_dim)

#         self.max_action = max_action

#     def forward(self, state):
#         a = F.relu(self.l1(state))
#         a = F.relu(self.l2(a))
#         return self.max_action * torch.sigmoid(self.l3(a))


# class Critic(nn.Module):
#     def __init__(self, state_dim, action_dim):
#         super(Critic, self).__init__()

#         self.l1 = nn.Linear(state_dim + action_dim, 400)
#         self.l2 = nn.Linear(400, 300)
#         self.l3 = nn.Linear(300, 1)

#     def forward(self, state, action):
#         state_action = torch.cat([state, action], 1)

#         q = F.relu(self.l1(state_action))
#         q = F.relu(self.l2(q))

#         return self.l3(q)


# class DDPG(object):
#     def __init__(self, state_dim, action_dim, max_action):
#         self.actor = Actor(state_dim, action_dim, max_action)
#         self.actor_target = Actor(state_dim, action_dim, max_action)
#         self.actor_target.load_state_dict(self.actor.state_dict())
#         self.actor_optimizer = torch.optim.Adam(self.actor.parameters())

#         self.critic = Critic(state_dim, action_dim)
#         self.critic_target = Critic(state_dim, action_dim)
#         self.critic_target.load_state_dict(self.critic.state_dict())
#         self.critic_optimizer = torch.optim.Adam(self.critic.parameters())

#         self.max_action = max_action

#     def select_action(self, state):
#         state = torch.FloatTensor(state.reshape(1, -1))
#         return self.actor(state).cpu().data.numpy().flatten()

#     def update(self, state, action, reward, next_state, done):
#         # Update the agent's parameters based on the transition
#         pass
#         action = agent.select_action(state)
#         next_state = [val1, val2]
#         # Perform action in the environment
#         # next_state, reward, done, _ = env.step(action)

#         # Update agent
#         # Note: You would typically store these transitions in a replay buffer and sample from it
#         agent.update(state, action, reward, next_state, done)

#         # Update state
#         state = next_state

#         # Break if done
#         if done:
#             t=30

In [7]:
# # Close the existing socket connection if it is open
# if 's' in locals() and s.fileno() != -1:
#     s.close()

# Create a new socket and bind to the address and port
def websocket ():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((TCP_IP, TCP_PORT))
    print('Waiting for Simulink to start')
    s.listen(1)
    conn, addr = s.accept()
    return conn


# s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# s.bind((TCP_IP, TCP_PORT))
# print('Waiting for Simulink to start')
# s.listen(1)
# conn, addr = s.accept()


In [15]:
def rewardcal(x, u):
    V = x[0]
    IL = x[1]
    Q = 10*np.eye(2)  # State penalty matrix
    R = 1 
    reward = -np.linalg.norm(x - np.array([Vref, ILref]))**2 
    # reward = -np.linalg.norm(x - np.array([Vref, ILref]))**2 - u**2 * R
    return reward


def isdone(x, t):
    # Define the desirable band
    desirable_band = [4.8, 5.2]

    # Initialize the start time and t0
    t0 = None

    V = x[0]
    IL = x[1]
    
    # Check if the state is within the desirable band
    if V >= desirable_band[0] and V <= desirable_band[1]:
        # Check if t0 is None (first time in the band)
        if t0 is None:
            t0 = t
        # Check if the state has been within the desirable band for 0.5 seconds
        elif t - t0 >= 0.5:
            return True
    else:
        # Reset t0 if V gets out of the band
        t0 = None
    
    return False


In [21]:
# import sys

# sys.setrecursionlimit(10000)
# state_dim = 2
# action_dim = 1
# max_action = 1
# agent = DDPG(state_dim, action_dim, max_action)

# # Number of episodes
# num_episodes = 50

# # Loop over episodes
# for i_episode in range(num_episodes):

#     # Reset the environment and state
#     # state = env.reset()
#     t1 = threading.Thread(target=SimRun)
#     t1.start()
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         future2 = executor.submit(websocket)
#         conn = future2.result()
#     time = 0
#     state = np.array([0, 0])
#     # Loop over steps within this episode
#     while time<30:
#         send_data(conn, u)

#         val1, val2,time = receive_data(conn)
#         reward = rewardcal([val1, val2], u)
#         done = isdone([val1, val2], time)
#         # Select action
#         action = agent.select_action(state)
#         next_state = [val1, val2]
#         # Perform action in the environment
#         # next_state, reward, done, _ = env.step(action)
#         sys.maximum_recursion_depth = 10000
#         # Update agent
#         # Note: You would typically store these transitions in a replay buffer and sample from it
#         agent.update(state, action, reward, next_state, done)

#         # Update state
#         state = next_state

#         # Break if done
#         if done:
#             break

Waiting for Simulink to start


Exception in thread Thread-65 (SimRun):
Traceback (most recent call last):
  File "c:\Users\pvm8318\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\pvm8318\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\pvm8318\AppData\Local\Temp\ipykernel_32364\3591277765.py", line 6, in SimRun
  File "c:\Users\pvm8318\AppData\Local\Programs\Python\Python311\Lib\site-packages\matlab\engine\matlabengine.py", line 71, in __call__
    _stderr, feval=True).result()
                         ^^^^^^^^
  File "c:\Users\pvm8318\AppData\Local\Programs\Python\Python311\Lib\site-packages\matlab\engine\futureresult.py", line 67, in result
    return self.__future.result(timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pvm8318\AppData\Local\Programs\Python\Python311\Lib\site-packages\matlab\engine\fevalfuture.py", line 82, in result
 

In [ ]:
# num_episodes = 10

# # Run the episodes
for episode in range(num_episodes):
    t1 = threading.Thread(target=SimRun)
    t1.start()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future2 = executor.submit(websocket)
        conn = future2.result()
    time = 0
    while time<2:

        send_data(conn, u)

        val1, val2,time = receive_data(conn)


        x= np.array([val1,val2])
        u = control_law(x)


        print('Duty cycle is:', u)
        print('time is:', time)
    # Close the connection
    conn.close()

Waiting for Simulink to start
